In [3]:
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras.utils import to_categorical
import pandas as pd

In [2]:
direc = os.getcwd()

In [3]:
def dictionarize_data (directory): #"D:\TIMIT\TRAIN"
    data = {"signal_path" : [],
            "phn_path" : [],
            "wrd_path" : [],
            "txt_path" : []}
    for root, direc, file in os.walk(directory):
            for i in file:            
                ext = i.split('.')
                file_path = os.path.join(root, ext[0])
                wav = "WAV"
                if ext[1] == wav:
                    data["signal_path"].append(file_path + ".WAV")
                    data["phn_path"].append(file_path + ".PHN")
                    data["wrd_path"].append(file_path + ".WRD")
                    data["txt_path"].append(file_path + ".TXT")
    return data   

In [4]:
def num_labels(lim_vec,label_vec,beg,end): #resolve o problema de intervalos inter-rotulos
    i = 0
    while i < len(lim_vec) and not(lim_vec[i][0] <= beg <= lim_vec[i][1]):
        i+=1
    j = 0
    
    while j < len(lim_vec) - 1 and not(lim_vec[j][0] <= end <= lim_vec[j][1]):
        j+=1
    return i,j

In [5]:
def save_melspectrogram(janelas_vec, sample_rate , nome):
    n_fft = 2048
    janela = 512

    #calcula a short term fourier transform:
    stft = librosa.stft(janelas_vec, n_fft = n_fft, hop_length = janela)

    #calcula o valor absoluto da stft (spectogram):
    spectrogram = np.abs(stft)

    #calcula o melspectrogram:
    melspectrogram = librosa.feature.melspectrogram(y = janelas_vec, S = spectrogram ,sr = sample_rate, hop_length = janela, fmax = sample_rate/2)
    file_name = str(nome) 
    os.chdir("C://Users//rober//UTFPR//machine_learning//JANELAS")
    diretorio = os.getcwd()
    np.save(file_name,np.array(melspectrogram))
    path = os.path.join(diretorio, file_name)
    return path

In [6]:
def save_labels(labels_vec):
    os.chdir("D:\TIMIT\LABELS")
    np.save(label_vec,"rotulos.npy")


In [7]:
def divide_signal(data):   
    #divide o sinal em intervalos de 100 ms (com passo de ms) da um rotulo -- caso o valor não exista, preencher com 0 até que de 100 ms
    #prestar atenção à valores de intervalo que ficam entre dois (ou mais) rotulos
    #copiar duas vezes e dar os dois (ou mais) rotulos
    #fazer a transformada de fourier e tmb melspectrograma nessa função mesmo!!
    janelas_vec = []
    labels_vec = []
    cont = 0
    window_data = {"window_path": [],
                   "label":[]}
    result = "resultado"
    for values in data["phn_path"]:
        limites_vec = []
        fonemas_vec = []
        #carrega o arquivo .WAV do correspondente ao arquivo .PHN
        path = values.split(".")
        #print(path[0] + ".WAV")
        raw_signal, sample_rate = librosa.load(path[0] + ".WAV", sr = 22050) #sr a definir

        #permite a leitura do arquivo phn
        read = open(values, "r")

        #guarda em um vetor todas as linhas do arquivo .PHN
        lines = read.readlines()

        #define o valor limite que possui rótulos
        lin = lines[-1].strip()
        modules = lin.split(" ")
        limite_total = int(modules[1]) # após esse valor não existe mais rótulos

        #copia o vetor do sinal extraido do .WAV do arquivo até o limite total
        signal = raw_signal[:limite_total]

        #adiciona todos os fonemas(rótulos) e seus intervalos em uma lista
        for line in lines: 
            li = line.strip()
            data_phonema = li.split(" ")
            lim_chao = int(data_phonema[0]) #limite por baixo
            lim_teto = int(data_phonema[1]) #limite por cima
            fonema = data_phonema[2]
            limites = (lim_chao,lim_teto)
            limites_vec.append(limites)
            fonemas_vec.append(fonema)
        cont_2 = 0
        #separa o vetor em vetores menores (janelas copias)
        for i in range(0, limite_total, int(sample_rate / 20)): 
            end = i + int(sample_rate/10)
            
            if end < limite_total:
                janela = signal[i : end]
            else:
                janela = signal[i : limite_total-1]
                
            while len(janela) < (2**math.ceil(math.log(sample_rate/10, 2))): #se o tamanho do vetor for menor que a proxima potencia acima dele, adiciona 0 
                janela = np.append(janela, 0)
                
            inicio, final = num_labels(limites_vec, labels_vec, i,end) #registra a primeira e a ultima ocorrencia da janela
            
            vers = 0
            for j in range(inicio, final + 1):
                jan = np.array(janela)
                wind_path = save_melspectrogram(jan, sample_rate, (result +"_s"+str(cont) + "_j"+str(cont_2)+"_v"+str(vers)))
                os.chdir(direc)
                window_data["window_path"].append(wind_path)
                window_data["label"].append(fonemas_vec[j])
                vers +=1
            cont_2 += 1
        cont += 1
    os.chdir(direc)        
    return window_data

In [8]:
data = dictionarize_data("D://TIMIT//TRAIN")
window_dict = divide_signal(data)

In [9]:
def contains(vec, label):
    for v in vec:
        if v == label:
            return True
    return False

In [10]:
def count_labels(labels):
    vec = []
    contador = 0
    for i in labels:
        if not contains(vec,i):
            vec.append(i)
            contador += 1
    print(vec)
    return contador

In [11]:
def normalize_pixels(data):
    #pixels ficam entre 0 e 1
    data = data.astype('float32')/255.0
    return data

In [12]:
def categorize_labels(label):
    label = pd.get_dummies(label)
    return label    

In [13]:
def load_data(window_data):
    t_data = np.array(window_data["window_path"])
    train_data = []
    for t in t_data:
        train_data.append(np.load(t+'.npy'))
    train_data = np.array(train_data)
    train_label = window_data["label"]
    return train_data, train_label

In [14]:
def create_network():
    network = models.Sequential()
    # network layers:
    network.add(layers.Conv1D(32,3,activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform', input_shape = (128,9)))
    network.add(layers.Conv1D(32,3,activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
    network.add(layers.MaxPooling1D(2))
    network.add(layers.Conv1D(64,3,activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
    network.add(layers.Conv1D(64,3,activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
    network.add(layers.MaxPooling1D(2))
    network.add(layers.Conv1D(128,3,activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
    network.add(layers.Conv1D(128,3,activation = 'relu', padding = 'same', kernel_initializer = 'he_uniform'))
    network.add(layers.MaxPooling1D(2))
    network.add(layers.Flatten())
    network.add(layers.Dense(128,activation ='relu'))
    network.add(layers.Dense(61,activation ='softmax'))
    #network configurations (optimizer, loss and metrics functions):
    network.compile(optimizer = 'RMSprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])
    return network

In [ ]:
train_data, train_label = load_data(window_dict)
train_data = normalize_pixels(train_data)
train_label = categorize_labels(train_label)
network = create_network()
network.fit(train_data, train_label, epochs = 5, batch_size = 128)

In [17]:
t_set, l_set = load_data(window_dict)
os.chdir("C://Users//rober//UTFPR//machine_learning//janelas_certas")
diretorio = os.getcwd()
np.save('resultados',t_set)
os.chdir("C://Users//rober//UTFPR//machine_learning//rotulos")
diretorio = os.getcwd()
np.save('rotulos', l_set)


In [12]:
direc = "C://Users//rober//UTFPR//machine_learning//janelas_certas"
for i in os.listdir(direc):
    path_npy = direc+"//"+i
    print(path_npy)

C://Users//rober//UTFPR//machine_learning//janelas_certas//resultados.npy


In [20]:
def load_janelas(npy_path):
    janelas = np.load(npy_path)
    return janelas

In [21]:
def load_labels(label_path):
    labels = np.load(label_path)
    return labels